In [1]:
# @title Process OpenStreetMap Way IDs and Generate CSV Files
# ------------------------------------------------------------------
# This is the main part of the script.
# 1. Edit the `way_ids_to_process` list below.
# 2. Run this cell to generate your CSV files.
# 3. Find your files in the file browser on the left sidebar.
# ------------------------------------------------------------------

import csv
import requests
import json
import re
from google.colab import files

# --- ⬇️ INPUT YOUR WAY IDs HERE ⬇️ ---
# Define the list of Way IDs to process.
# You can add more comma-separated string IDs to this list.
# For example: ["1264415480", "1264552828"]
way_ids_to_process = ["1264415480", "1264552828"]
# --- ⬆️ NO NEED TO EDIT BELOW THIS LINE ⬆️ ---


def get_way_info(way_id):
    """
    Fetches basic information and tags for a given OSM Way ID.
    This helps verify the way exists and provides context.
    """
    api_url = f"https://www.openstreetmap.org/api/0.6/way/{way_id}.json"
    print(f"\nFetching info for Way ID: {way_id}...")

    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()
        way_element = data.get('elements', [{}])[0]
        tags = way_element.get('tags')

        if tags:
            print("Found the following information for this way:")
            for key, value in tags.items():
                print(f"  - {key}: {value}")
            return True
        else:
            print("Could not find any descriptive tags for this Way ID, but it exists.")
            return True

    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"Error: Way ID '{way_id}' not found.")
        else:
            print(f"An HTTP error occurred: {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred while fetching info: {e}")
        return False


def fetch_and_convert_osm_data(way_id):
    """
    Fetches all node data for a specific OpenStreetMap 'way' from the API,
    and saves the node ID, latitude, and longitude to a CSV file.
    The CSV filename is dynamically generated from all available tags and includes the Way ID.
    """
    api_url = f"https://www.openstreetmap.org/api/0.6/way/{way_id}/full.json"
    print(f"Fetching full node data from {api_url}...")

    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()
        print("Data fetched successfully. Processing nodes...")

        way_tags = None
        for element in data.get('elements', []):
            if 'nodes' in element and element.get('id') == int(way_id):
                way_tags = element.get('tags')
                break

        if way_tags:
            tag_parts = [f"{key}_{value}" for key, value in way_tags.items()]
            descriptive_name = "-".join(tag_parts)
            safe_name = re.sub(r'[^\w_.)( -]', '', descriptive_name).replace(' ', '_')
            csv_file_name = f"{safe_name}_{way_id}_nodes.csv"
        else:
            csv_file_name = f"way_{way_id}_nodes.csv"

        with open(csv_file_name, mode='w', newline='', encoding='utf-8') as csv_file:
            fieldnames = ['Node ID', 'Latitude', 'Longitude']
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            writer.writeheader()

            node_count = 0
            for element in data.get('elements', []):
                if element.get('type') == 'node' and 'lat' in element and 'lon' in element:
                    writer.writerow({
                        'Node ID': element['id'],
                        'Latitude': element['lat'],
                        'Longitude': element['lon']
                    })
                    node_count += 1

        print(f"\nSuccess! Found and wrote {node_count} nodes.")
        print(f"Successfully created CSV file: {csv_file_name}")

    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the API request: {e}")
    except json.JSONDecodeError:
        print("Failed to decode JSON from the response.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# --- Main script execution ---
if not way_ids_to_process or not way_ids_to_process[0]:
    print("The 'way_ids_to_process' list is empty. Please add at least one Way ID to the list at the top of this cell and run it again.")
else:
    print(f"Starting batch processing for {len(way_ids_to_process)} Way ID(s)...")
    for way_id in way_ids_to_process:
        print("-" * 40)
        if get_way_info(way_id):
            fetch_and_convert_osm_data(way_id)
        else:
            print(f"Skipping Way ID {way_id} due to an error.")
    print("-" * 40)
    print("Batch processing complete. Check the file browser on the left for your CSV files.")


Starting batch processing for 2 Way ID(s)...
----------------------------------------

Fetching info for Way ID: 1264415480...
Found the following information for this way:
  - foot: yes
  - highway: path
  - sac_scale: mountain_hiking
Fetching full node data from https://www.openstreetmap.org/api/0.6/way/1264415480/full.json...
Data fetched successfully. Processing nodes...

Success! Found and wrote 22 nodes.
Successfully created CSV file: foot_yes-highway_path-sac_scale_mountain_hiking_1264415480_nodes.csv
----------------------------------------

Fetching info for Way ID: 1264552828...
Found the following information for this way:
  - building: temple
  - historic: archaeological_site
  - name: Candi V Gedong Songgo
Fetching full node data from https://www.openstreetmap.org/api/0.6/way/1264552828/full.json...
Data fetched successfully. Processing nodes...

Success! Found and wrote 4 nodes.
Successfully created CSV file: building_temple-historic_archaeological_site-name_Candi_V_Gedon